In [ ]:
import pandas as pd
import os
import plotly.express as px


from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Connexion NeonDB

In [39]:
load_dotenv("secrets.env")

True

Récupération des paramètres NEON

In [ ]:
NEON_USER=os.getenv("NEON_USER")
NEON_PASSWORD=os.getenv("NEON_PASSWORD")
NEON_HOST=os.getenv("NEON_HOST")
NEON_DBNAME=os.getenv("NEON_DBNAME")

engine = create_engine(
    f"postgresql+psycopg2://{NEON_USER}:{NEON_PASSWORD}@{NEON_HOST}/{NEON_DBNAME}?sslmode=require"
)

## Requête essai

In [41]:
with engine.connect() as conn:
    df_villes = pd.read_sql("SELECT * FROM villes", engine)
    df_meteos = pd.read_sql("SELECT * from meteos", engine)
    df_hotels = pd.read_sql("SELECT * from hotels", engine)
    df_condition = pd.read_sql("SELECT * FROM condition", engine)
    

In [ ]:
display(df_villes.head(5))
display(df_hotels.head(5))
display(df_meteos.head(5))
display(df_condition.head(5))


# Top 5 des villes pour la semaine du 14 au 21 Novembre

In [ ]:
query = """
SELECT 
    v.nom,
    v.latitude,
    v.longitude,
    m.temperature,
    m.duree_soleil,
    m.vent,
    m.pluie,
	c.code,
    c.description temps,
    v.ville_id
FROM villes v
	INNER JOIN 
		(
			SELECT 
				ville_id, 
				avg(temperature) temperature, 
				avg(duree_ensoleillement) duree_soleil, 
				avg(vent) vent,
				avg(pluie)pluie, 
				round(avg(condition),0) condition
			FROM meteos
			GROUP BY
				ville_id
		) m on v.ville_id=m.ville_id
	INNER JOIN condition c ON m.condition=c.code
ORDER BY 
    c.code,
	m.duree_soleil DESC,
	m.temperature DESC,
	m.pluie,
	m.vent
;
"""

with engine.connect() as conn:
    result = conn.execute(text(query))
    top5_destinations_sql = pd.DataFrame(result.fetchall(), columns=result.keys())

top5 = top5_destinations_sql.head(5).copy()
top5["rang"] = range(1, len(top5) + 1)
top5

In [ ]:
top5["rang_str"] = top5["rang"].astype(str)

custom_colors = {
    "1": "blue",
    "2": "red",
    "3": "green",
    "4": "orange",
    "5": "purple"
}


fig = px.scatter_mapbox(
    top5,
    lat="latitude",
    lon="longitude",
    size="temperature",
    color="rang_str",  
    color_discrete_map=custom_colors,
    hover_name="nom",
    hover_data={
        "temperature": True,
        "duree_soleil": True,
        "vent": True,
        "pluie": True,
        "temps": True,
        "rang": False,
        "latitude": False,
        "longitude": False
    },
    mapbox_style="open-street-map",
    zoom=4,
    title="Classement des destinations selon la météo"
)

fig.update_layout(
    legend_title_text="Classement météo",
    width=550,   
    height=600,  
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5,
        font=dict(size=13, color="black"),
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1
    )
)

fig.show()


## 20 hotels et où ils sont

In [45]:
top5

,nom,latitude,longitude,temperature,duree_soleil,vent,pluie,code,temps,ville_id,rang,rang_str
0,Collioure,42.525050,3.083155,12.693421,29298.445250,31.058938,0.975000,2,Nuageux,28,1,1
1,Cassis,43.214036,5.539632,10.267872,26778.400125,37.981552,4.737501,2,Nuageux,20,2,2
2,La palud sur Verdon,43.779686,6.341385,5.122990,26000.339625,12.661177,4.725000,2,Nuageux,18,3,3
3,Aigues-Mortes,43.566152,4.191540,10.722876,24950.529625,25.509239,2.200000,2,Nuageux,26,4,4
4,Saintes-Maries-de-la-mer,43.451592,4.427720,10.719892,24915.940875,25.591512,2.662500,2,Nuageux,27,5,5


In [ ]:
top5_villes = top5['ville_id'].tolist()

for ville in top5_villes:
    query = f"""
        SELECT 
            h.name AS hotel,
            v.nom,
            h.latitude,
            h.longitude,
            CAST(REPLACE(SUBSTR(h.rating, 18), ',', '.') AS FLOAT) AS note,
            prix
        FROM villes v
        INNER JOIN hotels h ON h.ville_id = v.ville_id
        WHERE v.ville_id = {ville}
        ORDER BY note DESC
    """

    with engine.connect() as conn:
        result = conn.execute(text(query))
        top_hotels = pd.DataFrame(result.fetchall(), columns=result.keys())

    if top_hotels.empty:
        print(f"⚠️ Pas de données pour ville_id={ville}")
        continue

    top_hotels = top_hotels.dropna(subset=["note"])

    fig = px.scatter_map(
        top_hotels,
        lat="latitude",
        width=500,
        height=500,
        lon="longitude",
        hover_name="hotel",
        hover_data={"nom": True, "note": True, "prix":True, "latitude": False, "longitude": False},
        color="note",
        color_continuous_scale="Viridis",
        range_color=[7, 10], 
        size="note", 
        size_max=20, 
        zoom=9,      
        title=f"🏨 Hôtels à {top_hotels['nom'].iloc[0]}"
    )

    fig.update_layout(
        coloraxis_colorbar=dict(title="Note"),
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()
